In [17]:
def date_time_():
    
    DICT_NUMERO = {'ce':1, 'cette':1, 'un':1, 'une':1, 'deux':2, 'trois':3,\
                   'quatre':4, 'cinq':5, 'six':6, 'sept':7,\
              'huit':8, 'neuf':9, 'dix':10, 'onze':11, 'douze':12, 'treize':13,'quartoze':14,\
               'cinze':15, 'seize':16, 'dix-sept':17,'dix sept':17, 'dix huit':18, \
               'dix-huit':18, 'dix-neuf':19, 'vingt':20, 'ving et un':21, 'trente':30, 'quarante':40,\
                  'quarante cinq':45, 'soixante':60}
    DICT_TEMPS = {'heure':1,  'journée':24,  'jour':24, \
              'mois':30*24, 'semaine':7*24, \
              'an':24*365, 'année':24*365, 'minute': 1/60}
    DICT_COMPLET_TEMPS = {}
    for temps in DICT_TEMPS:
        for numéro in DICT_NUMERO:
            key1 = numéro + ' '+temps
            key2 = str(DICT_NUMERO[numéro]) + ' ' +temps
            value = DICT_NUMERO[numéro]*DICT_TEMPS[temps]
            DICT_COMPLET_TEMPS[key1] = value
            DICT_COMPLET_TEMPS[key2] = value
    DICT_FOR_SPECIAL_CASE = {'aujourd\'hui':8, 'hébdomadaire':7*24, 'annuel': 365*24, \
                           'hier': 24, 'avant-hier': 48, 'annuelle': 365*24 }
    DICT_COMPLET_TEMPS['#cas_particulier#'] = DICT_FOR_SPECIAL_CASE
    DICT_COMPLET_TEMPS['avant hier'] = 72
    DICT_COMPLET_TEMPS['semaine dernière'] = 24*7
    return DICT_COMPLET_TEMPS

def _find_right_time(question, Dict):
    
    list_ = ['?', '.', ' ', '\n', '!']
    for symbol in list_:
        question = question.lower().strip(symbol)
    list_word = question.split()
    if len(list_word) < 2:
        return None   
    for key in Dict:
        if key in question:
            return Dict[key]
    for word in DICT_COMPLET_TEMPS['#cas_particulier#']:
            return DICT_COMPLET_TEMPS['#cas_particulier#'][word]        
    return None

def _find_right_geography(question, Dict):
    """
    Nous cherchons une ville, un département dans le dictionnaire pour retourner soit
    ce nom de ce lieu, soit ses coordonnées correspondance. Le retour de ce donnée dépend 
    de quel façon nous allons traiter avec cette information.
    
    """
    list_ = ['?', '.', ' ', '\n', '!']
    for symbol in list_:
        question = question.lower().strip(symbol) 
    list_geography = []
    for word in question.split():
        if word in Dict:
            list_geography.append(word)
    if len(list_geography) >0:
        return list_geography
    else:
        return None


def list_variable():
    """
    list_var is to manually complete """
    list_var = []
    if len(list_var) >0:
        return list_var
    else:
        return None

def _normalized_table():
    """
    return a data frame with new name
    """
    import pandas as pd
    df = pd.read_csv('fleet_donnees.csv', sep = ';')
    variable_names = list_variable()
    if variable_names ==None:
        return df
    else:
        name = list(df)
        dict_name = {name[i]: variable_names[i] for i in range(len(name))}
        df = df.rename(index= str, columns = dict_name)
        return df

def find_right_time_from_table(time, df):
    """
    given time found from a question, match it with the time given in table
    This depend on format of time in table.
    
    """
    return None
    
    
def _get_data_from_table_(ids, variable, time):
    """
    temps est un couple de début et fin
    """
    df = _normalized_table()
    time = find_right_time_from_table(time, df)
    if variable is None:
        return "#TO_CONSTRUCT#"
        
    
    
    if variable =="Tension Batterie":
        return min(df['Tension Batterie'])
    time1 = time[0]
    time2 = time[1]
    if time1 is None:
        time1 = string_to_datetime(df.iloc[0]['Debut période'])
    if time2 is None:
        time2 = string_to_datetime(df.iloc[len(df)-1]['Fin Période'])
    if len(ids) !=0:
        df = df.loc[(df['ID Véhicule'] ==  int(ids))] 
    List = []
    for i in range(len(df)):
        if string_to_datetime(df.iloc[i]['Debut période']) >=time1 and \
           string_to_datetime(df.iloc[i]['Fin Période'])<=time2:
                List.append(i)

    list_data_to_return = df.iloc[List][variable]
    if len(list_data_to_return)>1:
        t = sum(list_string_to_float(list_data_to_return))/len(list_data_to_return)
    else:
        t= list_data_to_return
    return t
        
        

méthode dateparser.search.search_dates retourne correctement les données de types un nombre + temps, mais pas d'ajectif possesif comme "cette semaine", "ce jour", "cette année". Il nous faut tourner ces adjectifs en nombre avant de traiter ces questions.

In [3]:
import time
import re
from dateparser.search import search_dates
import pandas as pd
DICT_COMPLET_TEMPS = date_time_()
KEYWORD_TO_FUNCTION_DICT = {'#id#':'ID Véhicule','#km#':'Distance', '#batterie#':'Tension Batterie',\
                            '#vitesse#':'Vitesse Moyenne', '#carburant#': 'carburant', '#pneu#':'pneu',\
                           '#conduite#':'conduite', '#position#':'position'}
def answers_with_data(question, answer):
    data = _analyse_reponse(question, answer)
    if data is None:
        return answer
    else:
        index = 0
        for word in answer.split():
            if "#" in word:
                good_answer = re.sub(word, str(data[index]), answer)
                index +=1
        return good_answer
    
                
def _analyse_reponse(question, answer):
    """
    arg: question is the question of user, reponse is given by chatbot

    """
    if '#' not in answer:
        return None
    else:
        parameter = []
        for key in KEYWORD_TO_FUNCTION_DICT:
            if key in answer.split():
                parameter.append(key)
        times = _find_right_time(question, DICT_COMPLET_TEMPS)
        ids = find_ids(question)
        variables = _get_variable(answer)
        data = []
        if len(variables)==0:
            return None
        else:
            for variable in variables:
                data.append(_get_data_from_table(ids, variable, times))
            return data
            
def _get_variable(reponse):
    list_variable =[]
    for word in reponse.split():
        if "#" in word:
            list_variable.append(word)
    return [KEYWORD_TO_FUNCTION_DICT[variable] for variable in list_variable
               if variable in KEYWORD_TO_FUNCTION_DICT]

    
def _get_data_from_table(ids, variable, time):
    """
    temps est un couple de début et fin
    """
    df = pd.read_csv('fleet_donnees.csv', sep = ';')
    #time1 = string_to_datetime(time[0])
    #time2 = string_to_datetime(time[1])
    if variable is None:
        return "#TO_CONSTRUCT#"
    if variable =="Tension Batterie":
        return min(df['Tension Batterie'])
    time1 = time[0]
    time2 = time[1]
    if time1 is None:
        time1 = string_to_datetime(df.iloc[0]['Debut période'])
    if time2 is None:
        time2 = string_to_datetime(df.iloc[len(df)-1]['Fin Période'])
    if len(ids) !=0:
        df = df.loc[(df['ID Véhicule'] ==  int(ids))] 
    List = []
    for i in range(len(df)):
        if string_to_datetime(df.iloc[i]['Debut période']) >=time1 and \
           string_to_datetime(df.iloc[i]['Fin Période'])<=time2:
                List.append(i)

    list_data_to_return = df.iloc[List][variable]
    if len(list_data_to_return)>1:
        t = sum(list_string_to_float(list_data_to_return))/len(list_data_to_return)
    else:
        t= list_data_to_return
    return t
        
          
def find_ids(question):
    if 'tous' in question or 'toutes' in question or 'tout'in question or 'toute' in question:
        return None
    else:
        return str(input('indiquez l\'identifiant de ce véhicule:'))
    

def string_to_datetime(string):
    t = search_dates(string)
    t = t[0][1]
    t = time.mktime(t.timetuple())
    return t

def list_string_to_float(List_string):
    List = []
    for string in List_string:
        if type(string)==str:
            string = re.sub(',', '.', string)
        else:
            pass
        string2float  = float(string)
        List.append(string2float)
    return List
 
    
def find_good_time_in_table(begin_time, end_time):
    """
    begin_time and end_time is in float type
    return right begin_time and end_time in table""" 
    import pandas as pd
    df = pd.read_csv('fleet_donnees.csv', sep = ';')
    start_time_colone = df['Debut période']
    end_time_colone = df['Fin Période']
    start_time_table = []
    for i in start_time_colone:
        time,_=find_right_time(i)
        start_time_table.append(time)
    end_time_table = []
    for i in fin_time_colone:
        time, _ = find_right_time(i)
        end_time_table.append(time)
    start = start_time_table[-1]
    for i in start_time_table:
        if begin_time <= i:
            start = i
            break
    end = end_time_table[-1]
    for i in end_time_table:
        if end <=i:
            end = i
            break
    start = start_time_table.index(start)
    end =  end_time_table.index(end)
    start = start_time_colone[start]
    end = _time_colone[end]
    return start, end


/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [20]:
question = 'combien de véhicules qui roulent cette semaine'
a, b = find_right_time(question)

In [18]:
question = 'combien de véhicules qui roulent aujourd\'hui'
a1, b1 = find_right_time(question)

In [21]:
norm

7.514115106198523

In [19]:
(b1-a1)/(60*60*24)

7.514033025194098

In [9]:
question = 'combien de véhicules qui roulent hier'
find_right_time(question)

(1530144000.0, 1530792962.5088363)

In [87]:
import pandas as pd
df = pd.read_csv('fleet_donnees.csv', sep = ';')
list_name = 

indiquez l'identifiant de ce véhicule:


'12.33'

In [14]:
liste = list(df)
diction = {liste[i]: list_name[i] for i in range(16)}

In [59]:
DICT_COMPLET_TEMPS = date_time_()
question = 'combien de voiture aujourd\'hui'
for temps in DICT_COMPLET_TEMPS:
    if temps in question:
        print(DICT_COMPLET_TEMPS[temps])

12


In [16]:
def datetime_to_float(d):
    import datetime
    epoch = datetime.datetime.utcfromtimestamp(0)
    total_seconds =  (d - epoch).total_seconds()
    return total_seconds
def float_to_datetime(fl):
    return datetime.datetime.fromtimestamp(fl)

In [73]:
answers_with_data('aldlc mazfq', '"#check_batterie#"')

indiquez l'identifiant de ce véhicule:


'"#check_batterie#"'

In [51]:
import dateparser
from dateparser.search import search_dates

In [1]:
import parsedatetime

In [3]:
cal = parsedatetime.Calendar()

In [10]:
cal.parse(" je vais  chez un ami demain matin")

(time.struct_time(tm_year=2018, tm_mon=6, tm_mday=5, tm_hour=14, tm_min=28, tm_sec=49, tm_wday=1, tm_yday=156, tm_isdst=0),
 0)

In [11]:
cal.parse("combien de km cette semaine")

(time.struct_time(tm_year=2018, tm_mon=6, tm_mday=5, tm_hour=14, tm_min=29, tm_sec=17, tm_wday=1, tm_yday=156, tm_isdst=0),
 0)

In [12]:
cal.parse("cette voiture a roulé pendant 3 heures")

(time.struct_time(tm_year=2018, tm_mon=6, tm_mday=5, tm_hour=14, tm_min=30, tm_sec=3, tm_wday=1, tm_yday=156, tm_isdst=0),
 0)

In [14]:
cal.parse("combien de km mon véhicule a parouru pour pendant 3 jours")

(time.struct_time(tm_year=2018, tm_mon=6, tm_mday=11, tm_hour=14, tm_min=31, tm_sec=38, tm_wday=0, tm_yday=162, tm_isdst=-1),
 1)

In [2]:
import sys
!{sys.executable} -m pip uninstall pandas

Uninstalling pandas-0.20.3:
  Would remove:
    /Users/NhatMinh/anaconda3/lib/python3.6/site-packages/pandas
    /Users/NhatMinh/anaconda3/lib/python3.6/site-packages/pandas-0.20.3-py3.6.egg-info
Proceed (y/n)? ^C
Operation cancelled by user


In [1]:
import pandas as pd
df = pd.read_csv('fleet_donnees.csv', sep = ';')
df

ImportError: Missing required dependencies ['numpy']

In [281]:
times = find_right_time('combien voiture 6 mois')

In [321]:
def func(a):
    if a >1:
        return a*a
    a= a+5
    print('a')
    return a
print(func(2))
        

4


In [67]:
min(df['Tension Batterie'])

12.33

In [316]:
_get_data_from_table(73426, 'Vitesse Moyenne', times)

71.6575

In [63]:
question = 'distance de voiture afemjoegj'
answer = '#km# '
answers_with_data(question, answer)

indiquez l'identifiant de ce véhicule:714448


'2304.9516775 '

In [309]:
variable = 'Vitesse Moyenne'
List = [1, 2, 5]
list_data_to_return = df.iloc[List][variable]

In [314]:
type('abbc')==str

True